In [85]:
from parse import parse, compile
import re

In [86]:
def add(left, right):
    return f'[{left},{right}]'

assert add('[1,2]', '[[3,4],5]') == '[[1,2],[[3,4],5]]'

def parse_pair(pair):
    parse_compile = compile('[{left:d},{right:d}]')
    pared_pair = parse_compile.parse(pair)
    return (pared_pair['left'], pared_pair['right'])

In [87]:
def find_explode(snailfish):
    brackets = 0
    b_string = ''
    explode_start = None
    explode_end = None
    for ix, char in enumerate(snailfish):
        brackets += 1 if char == '[' else -1 if char == ']' else 0
        b_string += str(brackets)
        if brackets == 5 and explode_start is None:
            explode_start = ix
        if explode_start is not None and explode_end is None and char == ']':
            explode_end = ix
    return (explode_start, explode_end)

assert find_explode('[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]') == (4,8)
assert find_explode('[[[[0,7],4],[7,[[8,4],9]]],[1,1]]') == (16,20)
assert find_explode('[[[[0,7],4],[[7,8],[0,13]]],[1,1]]') == (None,None)

In [88]:
def explode_match(snailfish, number, reversed=False):
    match = re.search("(\d+)", snailfish)
    if match is not None:
        span = match.span()
        if not reversed:
            number = str(int(match.group(1)) + number)
        else:
            number = str(int(match.group(1)[::-1]) + number)[::-1]
        return snailfish[:span[0]] + str(number) + snailfish[span[1]:]
    return snailfish

def explode(snailfish, pair_index):
    ix_start, ix_end = pair_index
    left_snailfish = snailfish[:ix_start]
    pair = snailfish[ix_start:ix_end+1]
    right_snailfish = snailfish[ix_end+1:]
    left, right = parse_pair(pair)
    
    left_snailfish_reversed =  left_snailfish[::-1]
    left_snailfish = explode_match(left_snailfish_reversed, left, True)[::-1]
    
    right_snailfish = explode_match(right_snailfish, right)
    
    return left_snailfish + '0' + right_snailfish

assert explode('[[[[0,7],4],[7,[[8,4],9]]],[1,1]]', (16,20)) == '[[[[0,7],4],[15,[0,13]]],[1,1]]'
assert explode('[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]', (4,8)) == '[[[[0,7],4],[7,[[8,4],9]]],[1,1]]'
assert explode('[[[[4,0],[5,4]],[[7,7],[0,[6,7]]]],[10,[[11,9],[11,0]]]]', (26,30)) == '[[[[4,0],[5,4]],[[7,7],[6,0]]],[17,[[11,9],[11,0]]]]'

In [89]:
def find_split(snailfish):
    match = re.search("\d{2,}", snailfish)
    return match.span() if match is not None else (None,None)

assert find_split('[[[[0,7],4],[15,[0,13]]],[1,1]]') == (13,15)
assert find_split('[[[[0,7],4],[[7,8],[0,13]]],[1,1]]') == (22,24)
assert find_split('[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]') == (None,None)

In [90]:
def split(snailfish, span):
    left_snailfish  = snailfish[:span[0]]
    right_snailfish = snailfish[span[1]:]
    number = int(snailfish[span[0]:span[1]])
    left = number // 2
    right = number - left
    number = f'[{left},{right}]'
    return left_snailfish + number + right_snailfish

assert split('[[[[0,7],4],[15,[0,13]]],[1,1]]', (13,15)) == '[[[[0,7],4],[[7,8],[0,13]]],[1,1]]'
assert split('[[[[0,7],4],[[7,8],[0,13]]],[1,1]]', (22,24)) == '[[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]'

In [91]:
test = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

test = test.split('\n')

In [92]:
def main(homework):
    snailfish = homework[0]
    queue = homework[1:]
    while True:
        if find_explode(snailfish) != (None,None):
            snailfish = explode(snailfish, find_explode(snailfish))
        elif find_split(snailfish) != (None,None):
            snailfish = split(snailfish, find_split(snailfish))
        else:
            if len(queue) == 0:
                break
            snailfish = add(snailfish, queue[0])
            queue = queue[1:]
    return snailfish
            
assert main(test) == '[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]'

In [93]:
def magnitude(snailfish):
    while True:
        match = re.search("(\[\d+,\d+\])", snailfish)
        if match is None:
            return int(snailfish)
        span = match.span()
        group = match.group(1)
        left, right = parse_pair(group)
        snailfish = snailfish[:span[0]] + str(left * 3 + right * 2) + snailfish[span[1]:]

assert magnitude('[[1,2],[[3,4],5]]') == 143

# Part 1

In [94]:
with open('input.txt', 'r') as f:
    prod = f.read().split('\n')
magnitude(main(prod))

4243

# Part 2

In [72]:
import itertools
permutations = list(itertools.permutations(prod, 2))
largest_magnitude = 0
for permutation in permutations:
    largest_magnitude = max(largest_magnitude, magnitude(main(permutation)))
largest_magnitude

4701